In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./data/mlb-pitches.csv')
data.drop(columns = ['Unnamed: 0'], inplace = True) 

pd.set_option('max_columns', None)

print(data.shape)
#data.head(3)
#list(data.columns.values)
#data.info()

(708653, 86)


In [3]:
df = data[['pitcher', 'p_throws', 'pitch_type', 'release_speed', 
           'release_spin_rate', 'spin_axis', 'pfx_-x', 'pfx_z', 
           'release_extension', 'release_pos_x', 'release_pos_z', 'effective_speed', 
           'plate_x', 'plate_-x', 'plate_z', 'vx0', 'vy0', 'vz0', 
           'ax', 'ay', 'az', 'type', 'zone', 'balls', 'strikes', 
           'batter', 'stand', 'events', 'description', 'hit_location', 
           'bb_type', 'hc_x','hc_y', 'hit_distance_sc', 'launch_speed', 
           'launch_angle', 'launch_speed_angle', 'estimated_ba_using_speedangle', 
           'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 
           'babip_value', 'iso_value','outs_when_up', 'inning', 'game_pk', 
           'release_pos_y', 'if_fielding_alignment', 'of_fielding_alignment', 
           'pitch_number']].copy()

print(df.shape)
#df.head()

(708653, 50)


In [4]:
#df[df.columns[df.isnull().any()]].isnull().sum()
#df.dropna(inplace = True)
df.dropna(subset = ['release_extension', 'release_spin_rate', 'if_fielding_alignment'], 
          inplace = True)

# Fill nulls with none so they aren't dropped
df['bb_type'] = df['bb_type'].fillna('not_in_play').astype(str)
df['events'] = df['events'].fillna('none').astype(str)
df.fillna(value = 0, inplace = True)


df = pd.get_dummies(df, columns = ['p_throws', 'type', 'stand', 'events', 
                                   'description', 'bb_type', 'if_fielding_alignment', 
                                   'of_fielding_alignment'])

# Drop pitch types Fasball, knuckleball, eephus, and screwball
df['pitch_type'] = df['pitch_type'].map({'FS': 8, 'CH': 7, 'KC': 6, 'CU': 5,
                                         'SL': 4, 'FC': 3, 'SI': 2, 'FF': 1})

print(df.shape)
df.head()

(702437, 109)


,pitcher,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,effective_speed,plate_x,plate_-x,plate_z,vx0,vy0,vz0,ax,ay,az,zone,balls,strikes,batter,hit_location,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,launch_speed_angle,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,outs_when_up,inning,game_pk,release_pos_y,pitch_number,p_throws_L,p_throws_R,type_B,type_S,type_X,stand_L,stand_R,events_catcher_interf,events_caught_stealing_2b,events_caught_stealing_3b,events_caught_stealing_home,events_double,events_double_play,events_field_error,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_game_advisory,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_none,events_other_out,events_passed_ball,events_pickoff_1b,events_pickoff_2b,events_pickoff_3b,events_pickoff_caught_stealing_2b,events_pickoff_caught_stealing_3b,events_sac_bunt,events_sac_bunt_double_play,events_sac_fly,events_sac_fly_double_play,events_single,events_stolen_base_2b,events_strikeout,events_strikeout_double_play,events_triple,events_triple_play,events_walk,events_wild_pitch,description_ball,description_blocked_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_bunt,description_foul_pitchout,description_foul_tip,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_pitchout,description_swinging_strike,description_swinging_strike_blocked,bb_type_fly_ball,bb_type_ground_ball,bb_type_line_drive,bb_type_not_in_play,bb_type_popup,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,519293,1,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80,92.8,-0.69,0.69,2.83,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,1.0,1,2,596019,5.0,77.54,155.31,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,2,9,632254,54.03,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
1,519293,4,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64,81.2,-0.71,0.71,2.62,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,4.0,1,1,596019,0.0,0.00,0.00,108.0,75.3,75.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.15,3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
2,519293,5,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88,75.2,-0.04,0.04,2.46,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,5.0,1,0,596019,0.0,0.00,0.00,157.0,83.5,65.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.34,2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,519293,5,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83,74.5,2.10,-2.10,3.89,2.375830,-109.205830,2.277617,-5.902656,19.427562,-38.284747,12.0,0,0,596019,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.61,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
4,519293,1,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66,90.9,-0.31,0.31,2.80,-5.868477,-132.500539,-6.486796,8.700586,30.117690,-15.941174,4.0,1,0,607043,4.0,144.83,145.72,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,1,9,632254,54.15,2,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702437 entries, 0 to 708651
Columns: 109 entries, pitcher to of_fielding_alignment_Strategic
dtypes: float64(33), int64(9), uint8(67)
memory usage: 275.3 MB


# Pitch Numbers

1. 4-seam fastball
2. Sinker
3. Cutter
4. Slider
5. Curveball
6. Knuckle Curve
7. Changeup
8. Split-Finger

### # of Pitches

In [6]:
df.pitch_type.value_counts()

1    248739
4    135610
2    108683
7     79978
5     58444
3     46575
6     13586
8     10822
Name: pitch_type, dtype: int64

### Velocity

In [7]:
df.groupby('pitch_type')['release_speed'].mean()

pitch_type
1    93.701701
2    93.002682
3    88.532404
4    84.661462
5    78.743339
6    81.071603
7    84.815458
8    85.706644
Name: release_speed, dtype: float64

### Spin Rate

In [8]:
df.groupby('pitch_type')['release_spin_rate'].mean()

pitch_type
1    2273.273680
2    2123.278866
3    2371.815674
4    2416.451714
5    2506.650862
6    2536.233770
7    1749.841731
8    1365.298743
Name: release_spin_rate, dtype: float64

### Spin Axis

In [9]:
df.groupby('pitch_type')['spin_axis'].mean()

pitch_type
1    192.876497
2    194.738506
3    174.087815
4    145.986594
5    124.633666
6     77.354924
7    195.125722
8    227.108575
Name: spin_axis, dtype: float64

### HB

In [10]:
df.groupby('pitch_type')['pfx_-x'].mean()

pitch_type
1    2.851918
2    5.645708
3   -1.369095
4   -3.217669
5   -3.871916
6   -6.285912
7    3.620740
8    9.603515
Name: pfx_-x, dtype: float64

### VB

In [11]:
df.groupby('pitch_type')['pfx_z'].mean()

pitch_type
1    16.101541
2     9.225791
3     7.945734
4     1.718492
5    -8.751586
6   -11.299573
7     6.824918
8     3.874884
Name: pfx_z, dtype: float64

### Pitch Data for Model

In [12]:
pitch_data = df[['pitch_type', 'release_speed', 'release_spin_rate', 'spin_axis', 
                 'pfx_-x', 'pfx_z','release_extension', 'release_pos_x', 'release_pos_z']]

pitch_data.dropna(inplace = True)

print(pitch_data.shape)
pitch_data.head()

(702437, 9)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z
0,1,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80
1,4,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64
2,5,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88
3,5,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83
4,1,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66


## Model

In [13]:
X = pitch_data

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled[:,:] *= -1

pca = PCA().fit(X_scaled)
principal_components = pca.transform(X_scaled)

In [14]:
model = pd.DataFrame(data = principal_components, columns = X.columns)

print(model.shape)
model.head()

(702437, 9)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z
0,-0.701450,-2.168989,-1.194822,0.116048,-0.980204,0.540022,0.086886,0.199409,0.085245
1,0.082749,1.153434,-1.582572,0.862083,-0.402985,2.210138,0.539885,0.623422,0.192923
2,1.335203,2.099339,-2.210909,1.255871,-0.621416,2.074313,-0.125899,1.170483,0.654474
3,1.566172,2.114108,-2.132896,1.827230,-0.188650,2.013033,-0.204999,1.151302,0.645274
4,-0.487624,-2.013066,-1.238125,0.295443,-0.576893,0.309695,0.006177,0.325671,-0.034932


In [15]:
km = KMeans(n_clusters = 4,
            random_state = 0)

km.fit(model)
label = km.fit_predict(model)

print('label:', label)
print('Number of iterations:', km.n_iter_)
print('Number of features:', km.n_features_in_)
print('Number of clusters:', km.n_clusters)
print('Inertia:', km.inertia_)
#print('Silhouette Score:', silhouette_score(X_scaled, labels, metric = 'euclidean'))

label: [2 3 3 ... 1 1 1]
Number of iterations: 9
Number of features: 9
Number of clusters: 4
Inertia: 3404554.314881164


In [17]:
centroids = km.cluster_centers_
print("Predicted clusters to points: ", label[:10], '\n')
print('Location of centroids:', '\n')
print(centroids)

Predicted clusters to points:  [2 3 3 3 2 2 3 3 2 1] 

Location of centroids: 

[[ 2.24209936 -0.46921577  1.05614271  0.13336386 -0.40863832 -0.24952995
  -0.04869717  0.05326531 -0.0654493 ]
 [-1.55352726  0.1001686   0.43465657  0.15283002 -0.13485208 -0.09130211
  -0.02843869  0.06353375  0.01187463]
 [ 0.25457069 -1.22594143 -1.19344699 -0.50595391  0.45227221 -0.02107704
  -0.01467817 -0.07706152 -0.03750594]
 [ 0.58464563  2.09837895 -0.67922371  0.18395996  0.1954971   0.5656666
   0.151254   -0.11113396  0.10711649]]


### Cluster Labels

Label:

0. Breaking Balls (SL, CU, **KC**) 4.395
1. Fastballs (FF, SI, FC) 1.504
2. FB's & BB's (FC, SL, CU) 2.711
3. Changeups (CH, Splitter) 5.820

In [18]:
X['label'] = label
model['label'] = label

#model.groupby(by = 'label').mean().T
X.groupby(by = 'label').mean().T

label,0,1,2,3
pitch_type,4.395764,1.504183,2.711367,5.819987
release_speed,82.796212,93.580263,89.940711,83.619467
release_spin_rate,2500.895909,2259.085483,2121.583370,2004.909114
spin_axis,74.719685,211.933129,143.154729,261.601022
pfx_-x,-7.917524,8.244385,-8.727942,10.304421
pfx_z,-2.787469,13.682036,11.676532,1.778995
release_extension,6.268315,6.376602,6.260798,6.250156
release_pos_x,-1.810462,-1.797599,1.926315,-0.420702
release_pos_z,5.825160,5.863431,5.917037,5.787869
